In [ ]:
!pip install sklearn

In [1]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random
import math
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
GB = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
#The variable GB is the memory size you want to use.
config = [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1500*GB))]
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], config)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
np.set_printoptions(threshold=40*40)


In [3]:
def load_data(res, source_labels, dataset='train', size=1., verbosity = False):
    data = np.zeros((0, res, res, res))
    labels = []
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        file = h5py.File(filename, 'r')
        tensor_len = len(file['tensor'])
        picked_ids = random.sample(range(tensor_len), math.ceil(tensor_len * size))
        if verbosity:
            print(filename)
            print('Picking ' + str(math.ceil(tensor_len * size)) + ' from ' + str(len(file['tensor'])) )
        data = np.concatenate((data, [file['tensor'][index] for index in picked_ids]))
        labels = np.concatenate((labels, np.full(len(picked_ids), i)))
        file.close()
        gc.collect()
    rng_state = np.random.get_state()
    np.random.shuffle(data)
    np.random.set_state(rng_state)
    np.random.shuffle(labels)
    return (data, labels)


In [ ]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(res, res, res)),
        keras.layers.Dense(res*2, activation='relu'),
        keras.layers.Dense(res/2, activation='sigmoid'),
        keras.layers.Dropout(.2, input_shape=(2,)),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return data
model_type = 'dense'
prepare_model(24, 20, 1)

In [ ]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Conv3D(
                res, 
                kernel_size=(6),
                strides=(2),
                activation='relu', 
                kernel_initializer='he_uniform', 
                data_format="channels_last",
                input_shape=(res, res, res, 1)
            ),        
        keras.layers.Conv3D(
                res, 
                kernel_size=(5),
                strides=(2),
                activation='relu', 
                kernel_initializer='he_uniform'
            ),        
        #keras.layers.Conv3D(
        #        res, 
        #        kernel_size=(2),
        #        activation='relu', 
        #        kernel_initializer='he_uniform'
        #    ),        
   
        #keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(res/2, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes, res):
    return data.reshape(len(data), res, res, res, 1)
model_type = 'conv3d'
prepare_model(24, 10, 1)

In [ ]:
def train(model, training, training_labels, res, num_classes, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if verbosity == 1:
        print("Log file " + log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 2 * num_classes
    
#    model.fit(
#        keras.utils.to_categorical(training, num_clases), 
#        training_labels, 
#        epochs=epochs)
    
    model.fit(
        prepare_data(training, num_classes, res), 
        training_labels, 
        epochs=epochs,
        verbose=verbosity,
        callbacks=[tensorboard_callback],
        validation_split=validation_split
    )

In [ ]:
#Confusion Matrix and Classification Report
def print_confusion_matrix(test, test_labels, base_labels, res):
    Y_pred = model.predict_generator(prepare_data(test, len(base_labels), res), len(test))
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(test_labels, y_pred))
    print('Classification Report')
    print(classification_report(test_labels, y_pred, target_names=base_labels))

In [4]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']
#num_classes = len(base_labels)

In [6]:
res = 24
training, training_labels = load_data(res, all_labels, 'train', 1, False)


In [7]:
training[0].shape

(24, 24, 24)

In [ ]:
subclases = [40]
resolutions = [24]#, 32]#, 48]

for j, num_classes in enumerate(subclases):
    base_labels = all_labels[:num_classes]
    for i,res in enumerate(resolutions):
        training, training_labels = load_data(res, base_labels, 'train', 1, False)
        model = prepare_model(res, num_classes, 1)
        model_name = 'models/' + model_type + '-' + str(num_classes) + '-' + str(res) + '.h5'
        train(model, training, training_labels, res, num_classes, 2)
        model.save(model_name)
        test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
        test_loss, test_acc = model.evaluate(
            prepare_data(test, num_classes, res),  
            test_labels, 
            verbose=2
        )
        print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
        print_confusion_matrix(test, test_labels, base_labels, res)
        gc.collect()

In [ ]:
base_labels = all_labels
res=24
training, training_labels = load_data(res, base_labels, 'train', 1, False)

In [ ]:
for i in range(40):
    print(str(i) + ':' + str(len(training_labels[training_labels == i])))


In [ ]:
model = keras.models.load_model('models/dense-20-24')


In [ ]:
num_classes = 20
res = 24
base_labels = all_labels[:num_classes]
test, test_labels = load_data(res, base_labels, 'test', 1.0, False)


In [ ]:
test_loss, test_acc = model.evaluate(
            prepare_data(test, num_classes),  
            test_labels, 
            verbose=2
        )
print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
print_confusion_matrix(test, test_labels, base_labels)

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()

In [ ]:
if not len(test):
    test, test_labels = load_data(dim, base_labels, 'test', 100)
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
p = model.predict(test)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index

In [ ]:
num_classes = 3
base_labels = all_labels[:num_classes]
num_classes = num_classes
res = 24
training, training_labels = load_data(res, base_labels, 'train', 1, True)


In [ ]:
print(len(training))
model = prepare_model(res, num_classes, 1)
train(model, training, training_labels, res, num_classes, 0)


In [ ]:
print(base_labels)
test, test_labels = load_data(res, base_labels, 'test', 1.0, True)
print(len(test))
test_loss, test_acc = model.evaluate(
    test,  
    test_labels, 
    verbose=2
)


In [ ]:
p = model.predict(test)
for index in range(len(p)):
    predicted_index = np.argmax(p[index])
    expected = base_labels[int(test_labels[index])]
    predicted = base_labels[predicted_index]
    if expected != predicted:
        draw(test[index], expected, predicted)
